In the following notebook, I will be building a car price predictor using multiple linear regression.

**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import yaml
import os

**Set Notebook Preferences**

In [2]:
#Set pandas preferences
pd.set_option('display.max_columns', 100)

#Set visualization preferences
plt.style.use('ggplot')

fontdict= {'title_fontsize':16,
           'label_fontsize':12}

**Read-in Data**

In [3]:
#Change directory to my_config.yaml
os.chdir('C:/Users/kishe/Documents/Data Science/Projects/Python/In Progress/Car_Price_Prediction')

#Read yaml file
from box import Box

with open('my_config.yaml') as file:
    yaml_data= Box(yaml.safe_load(file))

#Read in processed data
processed_data= pd.read_csv(yaml_data.base.path.processed_data, index_col=0,
                           dtype= {'symboling':'category'})

**Preview Data**

In [4]:
#Display head and data shape
print('Data shape:', processed_data.shape)
display(processed_data.head())

Data shape: (205, 20)


,symboling,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carwidth,carheight,enginetype,cylindernumber,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,highwaympg,price
0,0,gas,std,two,sedan,rwd,front,102.0,70.6,47.8,ohcv,twelve,mpfi,3.54,2.76,11.5,262,5000,17,1.589962
1,0,gas,std,four,sedan,rwd,front,120.9,71.7,56.7,ohcv,eight,mpfi,3.80,3.35,8.0,184,4500,16,1.590133
2,1,gas,std,two,hardtop,rwd,front,112.0,72.0,55.4,ohcv,eight,mpfi,3.80,3.35,8.0,184,4500,16,1.590259
3,0,gas,std,four,sedan,rwd,front,113.0,69.6,52.8,dohc,six,mpfi,3.63,4.17,8.1,176,4750,19,1.589806
4,0,gas,std,four,sedan,rwd,front,113.0,69.6,52.8,dohc,six,mpfi,3.63,4.17,8.1,176,4750,19,1.589945


**Check Info**

In [5]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 205 entries, 0 to 204
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   symboling         205 non-null    category
 1   fueltype          205 non-null    object  
 2   aspiration        205 non-null    object  
 3   doornumber        205 non-null    object  
 4   carbody           205 non-null    object  
 5   drivewheel        205 non-null    object  
 6   enginelocation    205 non-null    object  
 7   wheelbase         205 non-null    float64 
 8   carwidth          205 non-null    float64 
 9   carheight         205 non-null    float64 
 10  enginetype        205 non-null    object  
 11  cylindernumber    205 non-null    object  
 12  fuelsystem        205 non-null    object  
 13  boreratio         205 non-null    float64 
 14  stroke            205 non-null    float64 
 15  compressionratio  205 non-null    float64 
 16  horsepower        205 non-

# Supervised Learning

**Import libraries**

In [10]:
#Import sklearn libaries
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import cross_val_predict

#Import py files
from src.Models.machine_learning import regression_metrics

**Prepare data**

In [7]:
#Split data into X, y
X= processed_data.drop('price', axis = 1)
y= processed_data['price']

#Check shapes
print("Training data shape:%s\nTest data shape:%s" %(X.shape,y.shape))

Training data shape:(205, 19)
Test data shape:(205,)


In [8]:
#Subset numeric and categorical features
num_features= list(X.select_dtypes(include= ['float64', 'int64']).columns)
cat_features= list(X.select_dtypes(exclude= ['float64', 'int64']).columns)

**Build Pipeline**

In [9]:
#Create transformers
num_transformer= Pipeline(steps= [('scaler', StandardScaler())])
cat_transformer= Pipeline(steps= [('encoder', OneHotEncoder(handle_unknown='ignore'))])

#Init preprocessor with transformers
preprocessor= ColumnTransformer(transformers=[('numerics', num_transformer, num_features),
                                              ('categoricals', cat_transformer, cat_features)])

#Append preprocessor to pipeline
base_regressor= Pipeline(steps= [('preprocessor', preprocessor),
                                 ('regressor', LinearRegression(n_jobs=-1))
                                ])

**Test Base Model**

In [13]:
#Import scoring module
from src.Models.machine_learning import regression_metrics

#Run base_regressor
BaseRegressorPredictions= cross_val_predict(estimator= base_regressor, X= X, y= y, 
                                            cv=5, n_jobs= -1)

#Score BaseRegressorPredictions
BaseRegressorMetrics= regression_metrics(BaseRegressorPredictions, y)
BaseRegressorMetrics

{'MAE': 0.0004971838569072468,
 'MAPE': 0.031317182782029884,
 'MSE': 4.024706734518456e-07,
 'RMSE': 0.0006344057640436802,
 'R2': 0.7760059239553811}

**Check errors**

In [30]:
#Import visualization module
from src.Visualization.visualize import EDA_Viz, set_aesthetics

#Create dictionary containing BaseRegressorPredictions and y
BaseRegressorDict= {'Predictions': BaseRegressorPredictions,
                   'Actual': list(y)}

#Create residuals df
BaseRegressorResids= pd.DataFrame(data=BaseRegressorDict, columns= ['Predictions', 'Actual'])

#Calculate residuals

In [25]:
#Init EDA_Viz object
BRResidsViz= EDA_Viz(BaseRegressorResids)

#View BaseRegressorResids distribution
BRResidsViz.histogram(feature= 'residuals')

ValueError: Could not interpret value `residuals` for parameter `x`

In [31]:
BaseRegressorResids

,Predictions,Actual
0,1.591272,1.589962
1,1.591377,1.590133
2,1.590069,1.590259
3,1.590610,1.589806
4,1.590610,1.589945
...,...,...
200,1.586101,1.584944
201,1.587355,1.587527
202,1.587355,1.587751
203,1.587355,1.588125
